<a href="https://colab.research.google.com/github/Nataliaa1994/Diabetes_prediction_neural-networks/blob/master/diabetes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
sns.set()
tf.__version__

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'2.3.0'

In [2]:
from google.colab import files

In [3]:
uploaded=files.upload()

Saving diabetes.csv to diabetes.csv


In [13]:
import io
raw_data = pd.read_csv(io.StringIO(uploaded['diabetes.csv'].decode('utf-8')))
raw_data = pd.read_csv('diabetes.csv')

**Inspect Data**

In [ ]:
raw_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.600000,0.627000,50,1
1,1,85,66,29,0,26.600000,0.351000,31,0
2,8,183,64,0,0,23.300000,0.672000,32,1
3,1,89,66,23,94,28.100000,0.167000,21,0
4,0,137,40,35,168,43.100000,2.288000,33,1


In [ ]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
raw_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.000000,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Glucose,768.000000,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
BloodPressure,768.000000,69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
SkinThickness,768.000000,20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
Insulin,768.000000,79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
BMI,768.000000,31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
DiabetesPedigreeFunction,768.000000,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age,768.000000,33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Outcome,768.000000,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


In [14]:
nulls_summary = pd.DataFrame(raw_data.isnull().any(), columns=['Nulls'])   
nulls_summary['Num_of_nulls [qty]'] = pd.DataFrame(raw_data.isnull().sum())   
nulls_summary['Num_of_nulls [%]'] = round((raw_data.isnull().mean()*100),2)   
print(nulls_summary) 

                          Nulls  Num_of_nulls [qty]  Num_of_nulls [%]
Pregnancies               False                   0          0.000000
Glucose                   False                   0          0.000000
BloodPressure             False                   0          0.000000
SkinThickness             False                   0          0.000000
Insulin                   False                   0          0.000000
BMI                       False                   0          0.000000
DiabetesPedigreeFunction  False                   0          0.000000
Age                       False                   0          0.000000
Outcome                   False                   0          0.000000


Split data into test and train
This is done randomly - 80% train, 20% test

Train - to train neurons in the model
Test - to verify that the model works correctly

In [15]:
train_dataset = raw_data.sample(frac=0.8, random_state=0)
test_dataset = raw_data.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 614
test_dataset length: 154


In [ ]:


train_dataset.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
661,1,199,76,43,0,42.900000,1.394000,22,1
122,2,107,74,30,100,33.600000,0.404000,23,0
113,4,76,62,0,0,34.000000,0.391000,25,0
14,5,166,72,19,175,25.800000,0.587000,51,1
529,0,111,65,0,0,24.600000,0.660000,31,0


In [ ]:


test_dataset.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
9,8,125,96,0,0,0.000000,0.232000,54,1
11,10,168,74,0,0,38.000000,0.537000,34,1
19,1,115,70,30,96,34.600000,0.529000,32,1
23,9,119,80,35,0,29.000000,0.263000,29,1
28,13,145,82,19,110,22.200000,0.245000,57,0


In [16]:
train_stats = train_dataset.describe()
train_stats.pop('Outcome')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Pregnancies,614.000000,3.863192,3.382738,0.000000,1.000000,3.000000,6.000000,17.000000
Glucose,614.000000,121.636808,32.016768,0.000000,100.000000,118.500000,140.750000,199.000000
BloodPressure,614.000000,68.576547,19.911136,0.000000,62.000000,72.000000,80.000000,122.000000
SkinThickness,614.000000,20.314332,16.082796,0.000000,0.000000,23.000000,32.000000,99.000000
Insulin,614.000000,78.301303,115.537423,0.000000,0.000000,7.000000,127.750000,846.000000
BMI,614.000000,32.106678,7.712639,0.000000,27.400000,32.150000,36.575000,59.400000
DiabetesPedigreeFunction,614.000000,0.475573,0.329461,0.078000,0.245250,0.377000,0.639250,2.420000
Age,614.000000,33.364821,11.593037,21.000000,24.000000,29.000000,41.000000,72.000000


In [17]:
train_labels = train_dataset.pop('Outcome')
test_labels = test_dataset.pop('Outcome')

In [18]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [19]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [20]:
normed_train_data.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [21]:
normed_train_data.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
661,-0.846413,2.416334,0.372829,1.410555,-0.677714,1.399433,2.787664,-0.980314
122,-0.550794,-0.457161,0.272383,0.602238,0.187807,0.193620,-0.217244,-0.894056
113,0.040443,-1.425403,-0.330295,-1.263109,-0.677714,0.245483,-0.256702,-0.721538
14,0.336061,1.385624,0.171937,-0.081723,0.836947,-0.817707,0.338209,1.521187


In [22]:


normed_test_data.head(4)



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
9,1.222917,0.105045,1.377292,-1.263109,-0.677714,-4.162865,-0.739308,1.779963
11,1.814154,1.448091,0.272383,-1.263109,-0.677714,0.764112,0.186446,0.054790
19,-0.846413,-0.207292,0.071490,0.602238,0.153186,0.323277,0.162164,-0.117728
23,1.518535,-0.082357,0.573722,0.913129,-0.677714,-0.402803,-0.645215,-0.376504


In [23]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values
len(normed_test_data)

154

Model architecture

Describe the layers - number of layers, what layers included (hidden, softmax...), the type of model sequential


A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

In [24]:
def build_model():
    model = Sequential()
    model.add(Dense(250, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(2, activation='softmax'))
   
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [25]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 250)               2250      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 502       
Total params: 2,752
Trainable params: 2,752
Non-trainable params: 0
_________________________________________________________________


**Training the model**

In [26]:
history = model.fit(normed_train_data, train_labels.values, epochs=100, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.7318 - accuracy: 0.7434 - val_loss: 0.7036 - val_accuracy: 0.7154
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.7882 - val_loss: 0.6576 - val_accuracy: 0.6992
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5834 - accuracy: 0.7902 - val_loss: 0.6455 - val_accuracy: 0.7073
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5605 - accuracy: 0.7923 - val_loss: 0.6391 - val_accuracy: 0.7073
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5436 - accuracy: 0.7821 - val_loss: 0.6336 - val_accuracy: 0.6992
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5287 - accuracy: 0.7923 - val_loss: 0.6294 - val_accuracy: 0.7073
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5158 - accuracy: 0.7923 - val_loss: 0.6199 - val_accuracy: 0.7154
Epoch 8/100


In [27]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.731811,0.743381,0.703640,0.715447,0
1,0.621511,0.788187,0.657594,0.699187,1
2,0.583420,0.790224,0.645525,0.707317,2
3,0.560528,0.792261,0.639138,0.707317,3
4,0.543628,0.782077,0.633578,0.699187,4
...,...,...,...,...,...
95,0.387718,0.851324,0.569039,0.715447,95
96,0.386579,0.851324,0.560152,0.715447,96
97,0.387003,0.847251,0.553961,0.715447,97
98,0.386275,0.843177,0.565003,0.715447,98


At first I used 22 epochs and the batch size smaller than 15. Unfortunately the val_loss during training was too large a value and the accuracy too small. Both these attributes contribute to a bad model.

To train a good model it is important to make sure that the loss (incl. val_loss) is as small as possible, and accuracy (incl. val_accuracy) is as large as large as possible. It is also important that the difference between accuracy and val_accuracy is minimal, as if the accuracy is high but the val_accuracy is low, it leads to an overfitted model.

A value of 100 epochs and a batch size of 32 was ultimately chosen to train the model as the increased number of epochs helped optimise the model training.

In [28]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

The figures show a small difference between accuracy and val_accuracy as well as a relatively low val_loss (around 0.5 when originally the value was >1).

--------------------------------------------------------------------------------

**Validating the model**

Inputting test data into the model, and comparing the output from the model to the true expected output in outcome

Model outputs confidence levels on 2 nerons - 1) does not have diabetes (ie. 0), 2) has diabetes (ie. 1)

Whichever neuron has a higher confidence level, that is the suggested output of the model (ie. if [0.9883427   , 0.011657268 ], then the model is 98% confident in outcome of neuron 1 - the subject does not have diabetes)

Convert the confidence scores into a true or false output from the model to compare with true outcome data, and check the accuracy of the model on the test dataset

In [29]:
test_predictions = model.predict(normed_test_data)#.flatten()


In [30]:
test_predictions

array([[0.988457     , 0.0115429275 ],
       [0.06602439   , 0.9339756    ],
       [0.76135004   , 0.23865001   ],
       [0.78717923   , 0.21282074   ],
       [0.7205367    , 0.27946332   ],
       [0.9865587    , 0.013441241  ],
       [0.25142956   , 0.7485704    ],
       [0.19246839   , 0.80753165   ],
       [0.9464123    , 0.053587645  ],
       [0.1228005    , 0.87719953   ],
       [0.9819671    , 0.018032862  ],
       [0.07467404   , 0.92532593   ],
       [0.5997707    , 0.40022925   ],
       [0.37432992   , 0.6256701    ],
       [0.8766296    , 0.123370364  ],
       [0.2719116    , 0.72808844   ],
       [0.9527334    , 0.047266655  ],
       [0.6483325    , 0.35166746   ],
       [0.25537136   , 0.74462867   ],
       [0.7912027    , 0.20879732   ],
       [0.90281177   , 0.09718826   ],
       [0.78760993   , 0.21239012   ],
       [0.89298224   , 0.10701779   ],
       [0.28940275   , 0.7105973    ],
       [0.86503655   , 0.13496341   ],
       [0.6247966    , 0.

In [31]:
test_predictions_final = []
for entry in test_predictions:
  if entry[0] > entry[1]:
    test_predictions_final.append(0)
  else:
    test_predictions_final.append(1)

#test_predictions_final

In [32]:
test_labels_numpy = test_labels.to_numpy()

In [41]:
val_true = 0
val_false = 0
i = 0
for entry in test_predictions_final:
  if entry == test_labels_numpy[i]:
    val_true = val_true + 1
  else:
    val_false = val_false + 1
  i = i + 1

accuracy = val_true/len(test_predictions_final)

print(f' The model accuracy based on the test data is: {accuracy}' )

 The model accuracy based on the test data is: 0.7857142857142857


In [34]:
test_labels

9      1
11     1
19     1
23     1
28     0
      ..
746    1
753    1
754    1
759    1
763    0
Name: Outcome, Length: 154, dtype: int64

In [37]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime as dt

!rm -rf ./logs
log_dir = 'logs/fit/' + dt.now().strftime('%Y%m%d-%H%M%S')
tensorboard = TensorBoard(log_dir=log_dir)

In [ ]:
model = build_model()
history = model.fit(normed_train_data, train_labels.values, epochs=12, validation_split=0.2, verbose=1, batch_size=22,  callbacks=[tensorboard])
%load_ext tensorboard
!tensorboard dev upload --logdir logs


Epoch 1/12
23/23 [==============================] - 0s 9ms/step - loss: 0.7594 - accuracy: 0.6660 - val_loss: 0.6878 - val_accuracy: 0.7561
Epoch 2/12
23/23 [==============================] - 0s 2ms/step - loss: 0.6257 - accuracy: 0.7882 - val_loss: 0.6376 - val_accuracy: 0.7480
Epoch 3/12
23/23 [==============================] - 0s 2ms/step - loss: 0.5727 - accuracy: 0.7963 - val_loss: 0.6230 - val_accuracy: 0.7154
Epoch 4/12
23/23 [==============================] - 0s 2ms/step - loss: 0.5448 - accuracy: 0.7984 - val_loss: 0.6186 - val_accuracy: 0.7236
Epoch 5/12
23/23 [==============================] - 0s 2ms/step - loss: 0.5250 - accuracy: 0.8045 - val_loss: 0.6075 - val_accuracy: 0.7154
Epoch 6/12
23/23 [==============================] - 0s 2ms/step - loss: 0.5124 - accuracy: 0.8024 - val_loss: 0.6045 - val_accuracy: 0.7236
Epoch 7/12
23/23 [==============================] - 0s 2ms/step - loss: 0.5010 - accuracy: 0.8024 - val_loss: 0.5955 - val_accuracy: 0.7154
Epoch 8/12
23/23 [==